In [1]:
# One run of test to deduplicate the bio_med_research dataset
import pandas as pd
import os
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import xml.etree.ElementTree as ET
import json
from tqdm import tqdm
import pickle

In [2]:
# if use colab, run this part
from google.colab import drive

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/bionlp')

Mounted at /content/drive


In [3]:
# go to model dir
os.chdir('MedImageInsights')

In [4]:
# set directory to deduplicate
directory = "../dataset/QAs"

In [5]:
# install necessary package
!pip install mup
!pip install fvcore

  Preparing metadata (setup.py) ... done
  Created wheel for mup: filename=mup-1.0.0-py3-none-any.whl size=23629 sha256=e4cab55faad3264f8157b0487ff7fe9edcca9ab558b7b77e5ca6a3f8217114bb
  Stored in directory: /root/.cache/pip/wheels/f4/c8/88/3c23a3d10c50053b6552d2d30aee5b53ba89a47f742420036c
Successfully built mup
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=13d70ab524179a7962f19f1bb555f2f06173cb106528ee2378688351e679cced
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31528 sha256=707a4314c848d5d1ff2459729f8f4f92074d0e2a7ad2b3c9c498540e6f5dd2ec
  Stored in directory:

In [6]:
# load model
from medimageinsightmodel import MedImageInsight

classifier = MedImageInsight(
    model_dir="2024.09.27",
    vision_model_name="medimageinsigt-v1.0.0.pt",
    language_model_name="language_model.pth"
)

classifier.load_model()

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Model loaded successfully on device: cuda


In [7]:
def parse_document_xml(file_path):
    """
    Parse the XML file into a structured pandas DataFrame.

    Args:
        file_path (str): Path to the XML file.

    Returns:
        pd.DataFrame: A DataFrame containing extracted data.
    """
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Initialize storage for parsed data
    data = []

    # Extract document-level information
    doc_id = root.attrib.get("id", None)
    source = root.attrib.get("source", None)
    url = root.attrib.get("url", None)

    # Extract focus information
    focus_elem = root.find("Focus")
    if focus_elem is not None:
        focus = focus_elem.text.strip() if focus_elem.text is not None else None
    else:
        focus = None

    # Extract UMLS annotations
    umls_elem = root.find("FocusAnnotations/UMLS")
    umls_cuis = []
    semantic_types = []
    semantic_group = None

    if umls_elem is not None:
        umls_cuis = [cui.text.strip() for cui in umls_elem.findall("CUIs/CUI")]
        semantic_types = [stype.text.strip() for stype in umls_elem.findall("SemanticTypes/SemanticType")]
        semantic_group_elem = umls_elem.find("SemanticGroup")
        semantic_group = semantic_group_elem.text.strip() if semantic_group_elem.text is not None else None

    # Extract QA pairs
    qa_pairs_elem = root.find("QAPairs")
    if qa_pairs_elem is not None:
        for qa_pair in qa_pairs_elem.findall("QAPair"):
            pid = qa_pair.attrib.get("pid", None)

            # Extract question details
            question_elem = qa_pair.find("Question")
            question_id = question_elem.attrib.get("qid", None) if question_elem.attrib is not None else None
            question_type = question_elem.attrib.get("qtype", None) if question_elem.attrib is not None else None
            question_text = question_elem.text.strip() if question_elem.text is not None else None

            # Extract answer details
            answer_elem = qa_pair.find("Answer")
            answer_text = "".join(answer_elem.itertext()).strip() if "".join(answer_elem.itertext()) is not None else None

            # Store the extracted data
            data.append({
                "doc_id": doc_id,
                "source": source,
                "url": url,
                "focus": focus,
                "umls_cuis": umls_cuis,
                "semantic_types": semantic_types,
                "semantic_group": semantic_group,
                "pid": pid,
                "question_id": question_id,
                "question_type": question_type,
                "question_text": question_text,
                "answer_text": answer_text
            })

    # Convert data to pandas DataFrame
    df = pd.DataFrame(data)
    return df

In [8]:
# loading dataset
def parse_xml(file):
    tree = ET.parse(file)
    root = tree.getroot()

    sentence_data = []
    for sentence in root.findall('sentence'):
        sentence_id = sentence.get('id')
        sentence_text = sentence.get('text')

        sentence_data.append({
            "sentence_id": sentence_id,
            "sentence_text": sentence_text
        })

    return pd.DataFrame(sentence_data)


def load_dataset(path, filetype = "csv"):
    if filetype == "csv":
        all_files = []
        for root, dirs, files in tqdm(os.walk(path), desc = "Loading CSV files"):
            for file in tqdm(files, desc = "Processing file"):
                if file.endswith(".csv"):
                    all_files.append(os.path.join(root, file))
        ds = {}
        for f in all_files:
            df = pd.read_csv(f)
            ds[f] = df
        return ds
    elif filetype == "xml":
        all_files = []
        for root, dirs, files in tqdm(os.walk(path), desc = "Loading XML files"):
            for file in tqdm(files, desc = "Processing file"):
                if file.endswith(".xml"):
                    all_files.append(os.path.join(root, file))
        ds = {}
        for f in all_files:
            ds[f] = parse_document_xml(f)
        return ds
    elif filetype == "jsonl":
        all_files = []
        for root, dirs, files in tqdm(os.walk(path), desc = "Loading JSONL files"):
            for file in tqdm(files, desc = "Processing file"):
                if file.endswith(".jsonl"):
                    all_files.append(os.path.join(root, file))
        ds = {}
        for f in all_files:
            print("current file: ", f)
            with open(f, "r") as file:
                data = [json.loads(line) for line in file]
            ds[f] = pd.DataFrame(data)
        return ds
    elif filetype == "json":
        all_files = []
        for root, dirs, files in tqdm(os.walk(path), desc = "Loading JSON files"):
            for file in tqdm(files, desc = "Processing file"):
                if file.endswith(".json"):
                    all_files.append(os.path.join(root, file))
        ds = {}
        for f in all_files:
            with open(f, "r") as file:
                data = json.load(file)
            ds[f] = pd.DataFrame(data)
        return ds



In [9]:
# functions for deduplication
def get_embeddings(texts, batch_size = 64):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc = "Generating embeddings"):
        batch_texts = texts[i:i+batch_size]
        embeddings.extend(classifier.encode(texts = batch_texts)['text_embeddings'])
    return np.array(embeddings)

def compute_similarity(embeddings, threshold = 0.9):
    # n = len(embeddings)
    # to_remove = set()
    # for i in tqdm(range(n), desc = "Computing similarity"):
    #     for j in range(i+1, n):
    #         sim = cosine_similarity(embeddings[i].reshape(1, -1), embeddings[j].reshape(1, -1))[0][0]
    #         if sim > threshold:
    #             to_remove.add(j)
    # return to_remove
    similarity_matrix = cosine_similarity(embeddings)
    np.fill_diagonal(similarity_matrix, 0)  # Ignore self-similarity

    # Find indices of pairs with similarity above the threshold
    to_remove = set()
    for i in range(similarity_matrix.shape[0]):
        if i in to_remove:
            continue
        similar_indices = np.where(similarity_matrix[i] > threshold)[0]
        to_remove.update(similar_indices)

    return to_remove

def compute_similarity_chunked(embeddings, threshold=0.9, chunk_size=8000):
    """
    Compute cosine similarity in chunks to reduce memory usage.
    """
    n = len(embeddings)
    to_remove = set()
    for i in tqdm(range(0, n, chunk_size), desc= "Calcuating Similarity"):
        # Get the current chunk
        chunk_embeddings = embeddings[i:i + chunk_size]

        # Compute cosine similarity for the current chunk against all embeddings
        similarity_matrix = cosine_similarity(chunk_embeddings, embeddings)

        # Iterate through the chunk rows to find high-similarity indices
        for row_idx, similarities in enumerate(similarity_matrix):
            actual_idx = i + row_idx  # Map back to the original index
            if actual_idx in to_remove:
                continue

            similar_indices = np.where(similarities > threshold)[0]
            similar_indices = [idx for idx in similar_indices if idx > actual_idx]  # Avoid duplicates
            to_remove.update(similar_indices)

    return to_remove

def compute_similarity_between_datasets(embeddings1, embeddings2, threshold = 0.9):
    to_remove = set()
    for i in tqdm(range(len(embeddings1)), desc = "Computing similarity"):
        for j in range(len(embeddings2)):
            sim = cosine_similarity(embeddings1[i].reshape(1, -1), embeddings2[j].reshape(1, -1))[0][0]
            if sim > threshold:
                to_remove.add(j)
    return to_remove

def compute_similarity_between_datasets_chunked(embeddings1, embeddings2, threshold=0.9, chunk_size1=8000, chunk_size2=8000):
    """
    Compute cosine similarity between two datasets in chunks to reduce memory usage.
    Removes entries from embeddings1 based on high similarity with embeddings2.
    """
    to_remove = set()
    n1, n2 = len(embeddings1), len(embeddings2)

    for i in tqdm(range(0, n1, chunk_size1), desc="Processing dataset1 in chunks"):
        # Get a chunk from embeddings1
        chunk_embeddings1 = embeddings1[i:i + chunk_size1]

        for j in range(0, n2, chunk_size2):
            # Get a chunk from embeddings2
            chunk_embeddings2 = embeddings2[j:j + chunk_size2]

            # Compute cosine similarity for the two chunks
            similarity_matrix = cosine_similarity(chunk_embeddings1, chunk_embeddings2)

            # Check rows in chunk_embeddings1 with high similarity to chunk_embeddings2
            for row_idx, similarities in enumerate(similarity_matrix):
                actual_idx = i + row_idx  # Map back to the original index in embeddings1
                if actual_idx in to_remove:
                    continue
                if np.any(similarities > threshold):
                    to_remove.add(actual_idx)

    return to_remove

def deduplication_within_dataset_qa(dataset, threshold = 0.9):
    questions = dataset["question"].tolist()
    #answers = dataset["answer"].tolist()

    question_embeddings = get_embeddings(questions)
    to_remove_questions = compute_similarity_chunked(question_embeddings, threshold)

    new_dataset = dataset.drop(index = list(to_remove_questions)).reset_index(drop=True)

    answers = new_dataset["answer"].tolist()
    answer_embeddings = get_embeddings(answers)
    to_remove_answers = compute_similarity_chunked(answer_embeddings, threshold)

    new_dataset = new_dataset.drop(index = list(to_remove_answers)).reset_index(drop=True)
    return new_dataset, list(to_remove_questions), list(to_remove_answers)


def deduplicate_across_datasets_qa(new_dataset, old_question_embeddings_saved, old_answer_embeddings_saved, threshold = 0.9):
    # Combine all old dataset questions and answers
    # all_old_questions = []
    # all_old_answers = []

    # for dataset in old_datasets:
    #     all_old_questions.extend(dataset["question"].tolist())
    #     all_old_answers.extend(dataset["answer"].tolist())

    # Generate embeddings for old dataset questions and answers
    # old_question_embeddings = get_embeddings(all_old_questions)
    # old_answer_embeddings = get_embeddings(all_old_answers)
    old_question_embeddings = []
    old_answer_embeddings = []
    for old_embed in old_question_embeddings_saved:
        old_question_embeddings.extend(old_embed)
    for old_embed in old_answer_embeddings_saved:
        old_answer_embeddings.extend(old_embed)

    # Generate embeddings for new dataset questions and answers
    new_question_embeddings = get_embeddings(new_dataset["question"].tolist())
    new_answer_embeddings = get_embeddings(new_dataset["answer"].tolist())

    # Deduplicate new questions
    to_remove_questions = compute_similarity_between_datasets_chunked(new_question_embeddings, old_question_embeddings)

    # Deduplicate new answers
    to_remove_answers = compute_similarity_between_datasets_chunked(new_answer_embeddings, old_answer_embeddings)

    # Combine removal indices
    to_remove = to_remove_questions.union(to_remove_answers)

    # Drop duplicates from new dataset
    deduplicated_new_dataset = new_dataset.drop(index=list(to_remove)).reset_index(drop=True)

    return deduplicated_new_dataset, list(to_remove_questions), list(to_remove_answers)



In [10]:
#deduplicated data loading
deduplicated_medicationqa = pd.read_csv("../deduplicated_data/QAs/MedicationQA/medicationqa_train_fulltext_deduplicated.csv")
deduplicated_pubmed1 = pd.read_csv("../deduplicated_data/QAs/PubMedQA/ori_pqaa_deduplicated.csv")
deduplicated_pubmed2 = pd.read_csv("../deduplicated_data/QAs/PubMedQA/ori_pqau_deduplicated.csv")
deduplicated_pubmed3 = pd.read_csv("../deduplicated_data/QAs/PubMedQA/ori_pqal_deduplicated.csv")
deduplicated_medmcqa_train = pd.read_csv("../deduplicated_data/QAs/MedMCQA/medmcqa_train_fulltext_deduplicated.csv")
deduplicated_medmcqa_dev = pd.read_csv("../deduplicated_data/QAs/MedMCQA/medmcqa_dev_fulltext_deduplicated.csv")
deduplicated_medmcqa_test = pd.read_csv("../deduplicated_data/QAs/MedMCQA/medmcqa_test_fulltext_deduplicated.csv")
deduplicated_medqa_train = pd.read_csv("../deduplicated_data/QAs/MedQA-USMLE/medqa_train_deduplicated.csv")
deduplicated_medqa_dev = pd.read_csv("../deduplicated_data/QAs/MedQA-USMLE/medqa_dev_deduplicated.csv")
deduplicated_medqa_test = pd.read_csv("../deduplicated_data/QAs/MedQA-USMLE/medqa_test_deduplicated.csv")
deduplicated_liveqa_test = pd.read_csv("../deduplicated_data/QAs/LiveQA/trec_qa_test_fulltext_deduplicated.csv")
deduplicated_liveqa_train1 = pd.read_csv("../deduplicated_data/QAs/LiveQA/trec_qa_train1_fulltext_deduplicated.csv")
deduplicated_liveqa_train2 = pd.read_csv("../deduplicated_data/QAs/LiveQA/trec_qa_train2_fulltext_deduplicated.csv")

## Deduplicate MedQuAD

In [12]:
medquad_ds = load_dataset("../dataset/QAs/MedQuAD", filetype = "xml")

Loading XML files: 0it [00:00, ?it/s]
Processing file: 100%|██████████| 3/3 [00:00<00:00, 29817.33it/s]

Processing file: 100%|██████████| 981/981 [00:00<00:00, 558064.86it/s]

Processing file: 100%|██████████| 1086/1086 [00:00<00:00, 593564.52it/s]

Processing file: 100%|██████████| 99/99 [00:00<00:00, 444102.78it/s]

Processing file: 100%|██████████| 59/59 [00:00<00:00, 290450.63it/s]

Processing file: 100%|██████████| 4366/4366 [00:00<00:00, 676780.67it/s]
Loading XML files: 6it [00:00, 41.39it/s]
Processing file: 100%|██████████| 277/277 [00:00<00:00, 589757.47it/s]

Processing file: 100%|██████████| 116/116 [00:00<00:00, 276285.78it/s]

Processing file: 100%|██████████| 88/88 [00:00<00:00, 346897.32it/s]

Processing file: 100%|██████████| 48/48 [00:00<00:00, 87419.28it/s]

Processing file: 100%|██████████| 2685/2685 [00:00<00:00, 666609.82it/s]

Processing file: 100%|██████████| 1312/1312 [00:00<00:00, 663163.03it/s]
Loading XML files: 12it [00:00, 50.32it/s]
Processing file: 100%

In [13]:
for k in medquad_ds.keys():
    if not k.endswith(".xml"):
        print(k)

In [17]:
# deduplicate each dataset
for k in medquad_ds:
    # rename columns
    medquad_ds[k] = medquad_ds[k].rename(columns = {"question_text": "question", "answer_text": "answer"})
    ds_deduplicated, removed_questions, removed_answers = deduplication_within_dataset_qa(medquad_ds[k])
    print("For the dataset: ", k)
    print("Number of removed questions: ", len(removed_questions))
    print("Number of removed answers: ", len(removed_answers))
    medquad_ds[k] = ds_deduplicated

Calcuating Similarity: 100%|██████████| 1/1 [00:00<00:00, 763.29it/s]


For the dataset:  ../dataset/QAs/MedQuAD/4_MPlus_Health_Topics_QA/0000153.xml
Number of removed questions:  0
Number of removed answers:  0


Calcuating Similarity: 100%|██████████| 1/1 [00:00<00:00, 961.11it/s]


For the dataset:  ../dataset/QAs/MedQuAD/4_MPlus_Health_Topics_QA/0000691.xml
Number of removed questions:  0
Number of removed answers:  0


Calcuating Similarity: 100%|██████████| 1/1 [00:00<00:00, 633.48it/s]


...

Calcuating Similarity: 100%|██████████| 1/1 [00:00<00:00, 1038.71it/s]


For the dataset:  ../dataset/QAs/MedQuAD/6_NINDS_QA/0000054.xml
Number of removed questions:  2
Number of removed answers:  0


Calcuating Similarity: 100%|██████████| 1/1 [00:00<00:00, 616.63it/s]


For the dataset:  ../dataset/QAs/MedQuAD/6_NINDS_QA/0000135.xml
Number of removed questions:  2
Number of removed answers:  0


KeyError: 'question'

In [ ]:
# self deduplicate first
trec_qa_train_1_self_dedup, removed_questions_self_train_1, removed_answers_self_train_1 = deduplication_within_dataset_qa(trec_qa_train_1)
print(len(removed_questions_self_train_1), len(removed_answers_self_train_1))
trec_qa_train_2_self_dedup, removed_questions_self_train_2, removed_answers_self_train_2 = deduplication_within_dataset_qa(trec_qa_train_2)
print(len(removed_questions_self_train_2), len(removed_answers_self_train_2))
trec_qa_test_self_dedup, removed_questions_self_test, removed_answers_self_test = deduplication_within_dataset_qa(trec_qa_test)
print(len(removed_questions_self_test), len(removed_answers_self_test))

Calcuating Similarity: 100%|██████████| 1/1 [00:00<00:00, 195.85it/s]


55 10


Calcuating Similarity: 100%|██████████| 1/1 [00:00<00:00, 155.92it/s]


0 4


Calcuating Similarity: 100%|██████████| 1/1 [00:00<00:00, 312.87it/s]

0 0


In [ ]:
old_questions = []
old_answers = []

with open("../deduplicated_embeddings/QAs/medicationqa_question_embeddings.pkl", "rb") as f:
    medication_qa_q_embed = pickle.load(f)
    old_questions.append(medication_qa_q_embed)

with open("../deduplicated_embeddings/QAs/medicationqa_answer_embeddings.pkl", "rb") as f:
    medication_qa_a_embed = pickle.load(f)
    old_answers.append(medication_qa_a_embed)

#pubmed1,2,3
with open("../deduplicated_embeddings/QAs/pubmed1_question_embeddings.pkl", "rb") as f:
    pubmed1_q_embed = pickle.load(f)
    old_questions.append(pubmed1_q_embed)

with open("../deduplicated_embeddings/QAs/pubmed1_answer_embeddings.pkl", "rb") as f:
    pubmed1_a_embed = pickle.load(f)
    old_answers.append(pubmed1_a_embed)

with open("../deduplicated_embeddings/QAs/pubmed2_question_embeddings.pkl", "rb") as f:
    pubmed2_q_embed = pickle.load(f)
    old_questions.append(pubmed2_q_embed)

with open("../deduplicated_embeddings/QAs/pubmed2_answer_embeddings.pkl", "rb") as f:
    pubmed2_a_embed = pickle.load(f)
    old_answers.append(pubmed2_a_embed)

with open("../deduplicated_embeddings/QAs/pubmed3_question_embeddings.pkl", "rb") as f:
    pubmed3_q_embed = pickle.load(f)
    old_questions.append(pubmed3_q_embed)

with open("../deduplicated_embeddings/QAs/pubmed3_answer_embeddings.pkl", "rb") as f:
    pubmed3_a_embed = pickle.load(f)
    old_answers.append(pubmed3_a_embed)

# medmcqa
with open("../deduplicated_embeddings/QAs/medmcqa_train_question_embeddings.pkl", "rb") as f:
    medmcqa_train_q_embed = pickle.load(f)
    old_questions.append(medmcqa_train_q_embed)

with open("../deduplicated_embeddings/QAs/medmcqa_train_answer_embeddings.pkl", "rb") as f:
    medmcqa_train_a_embed = pickle.load(f)
    old_answers.append(medmcqa_train_a_embed)

with open("../deduplicated_embeddings/QAs/medmcqa_dev_question_embeddings.pkl", "rb") as f:
    medmcqa_dev_q_embed = pickle.load(f)
    old_questions.append(medmcqa_dev_q_embed)

with open("../deduplicated_embeddings/QAs/medmcqa_dev_answer_embeddings.pkl", "rb") as f:
    medmcqa_dev_a_embed = pickle.load(f)
    old_answers.append(medmcqa_dev_a_embed)

with open("../deduplicated_embeddings/QAs/medmcqa_test_question_embeddings.pkl", "rb") as f:
    medmcqa_test_q_embed = pickle.load(f)
    old_questions.append(medmcqa_test_q_embed)

with open("../deduplicated_embeddings/QAs/medmcqa_test_answer_embeddings.pkl", "rb") as f:
    medmcqa_test_a_embed = pickle.load(f)
    old_answers.append(medmcqa_test_a_embed)

with open("../deduplicated_embeddings/QAs/medqa_train_question_embeddings.pkl", "rb") as f:
    medqa_train_a_embed = pickle.load(f)
    old_questions.append(medqa_train_a_embed)

with open("../deduplicated_embeddings/QAs/medqa_train_answer_embeddings.pkl", "rb") as f:
    medqa_train_a_embed = pickle.load(f)
    old_answers.append(medqa_train_a_embed)

with open("../deduplicated_embeddings/QAs/medqa_dev_question_embeddings.pkl", "rb") as f:
    medqa_dev_a_embed = pickle.load(f)
    old_questions.append(medqa_dev_a_embed)

with open("../deduplicated_embeddings/QAs/medqa_dev_answer_embeddings.pkl", "rb") as f:
    medqa_dev_a_embed = pickle.load(f)
    old_answers.append(medqa_dev_a_embed)

with open("../deduplicated_embeddings/QAs/medqa_test_question_embeddings.pkl", "rb") as f:
    medqa_test_a_embed = pickle.load(f)
    old_questions.append(medqa_test_a_embed)

with open("../deduplicated_embeddings/QAs/medqa_test_answer_embeddings.pkl", "rb") as f:
    medqa_test_a_embed = pickle.load(f)
    old_answers.append(medqa_test_a_embed)

In [ ]:
# deduplicate between existing dataset
trec_qa_train1_full, removed_questions_train, removed_answers_train = deduplicate_across_datasets_qa(trec_qa_train_1_self_dedup, old_questions, old_answers)
print(len(removed_questions_train), len(removed_answers_train))

Processing dataset1 in chunks: 100%|██████████| 1/1 [00:02<00:00,  2.72s/it]

1 0


In [ ]:
trec_qa_train1_full.to_csv("../deduplicated_data/QAs/LiveQA/trec_qa_train1_fulltext_deduplicated.csv", index = False)

In [ ]:
with open("../deduplicated_embeddings/QAs/trec_train1_question_embeddings.pkl", "rb") as f:
    trec_train1_q_embed = pickle.load(f)
    old_questions.append(trec_train1_q_embed)

with open("../deduplicated_embeddings/QAs/trec_train1_answer_embeddings.pkl", "rb") as f:
    trec_train1_a_embed = pickle.load(f)
    old_answers.append(trec_train1_a_embed)

In [ ]:
trec_qa_train2_full, removed_questions_train, removed_answers_train = deduplicate_across_datasets_qa(trec_qa_train_2_self_dedup, old_questions, old_answers)
print(len(removed_questions_train), len(removed_answers_train))

Processing dataset1 in chunks: 100%|██████████| 1/1 [00:02<00:00,  3.00s/it]

27 3


In [ ]:
trec_qa_train2_full.to_csv("../deduplicated_data/QAs/LiveQA/trec_qa_train2_fulltext_deduplicated.csv", index = False)

In [ ]:
with open("../deduplicated_embeddings/QAs/trec_train2_question_embeddings.pkl", "rb") as f:
    trec_train2_q_embed = pickle.load(f)
    old_questions.append(trec_train2_q_embed)

with open("../deduplicated_embeddings/QAs/trec_train2_answer_embeddings.pkl", "rb") as f:
    trec_train2_a_embed = pickle.load(f)
    old_answers.append(trec_train2_a_embed)

In [ ]:
trec_qa_test_full, removed_questions_test, removed_answers_test = deduplicate_across_datasets_qa(trec_qa_test_self_dedup, old_questions, old_answers)
print(len(removed_questions_test), len(removed_answers_test))

Processing dataset1 in chunks: 100%|██████████| 1/1 [00:02<00:00,  2.65s/it]

15 1


In [ ]:
trec_qa_test_full.to_csv("../deduplicated_data/QAs/LiveQA/trec_qa_test_fulltext_deduplicated.csv", index = False)